<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [60]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [61]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
#with open('../../data/hopsworks-api-key.txt', 'r') as file:
#    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1159325
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "iceland", "city": "reykjavik", "street": "husdyragardurinn", "aqicn_url": "http://api.waqi.info/feed/iceland/husdyrag.", "latitude": 64.15, "longitude": -21.94}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [81]:
# Retrieve feature groups
#air_quality_fg = fs.get_feature_group(
#    name='air_quality',
#    version=1,
#)
air_quality_with_lag_fg = fs.get_feature_group(
    name='air_quality_with_3day_lag',
    version=1,
)

weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)


---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [82]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,22.0,iceland,reykjavik,husdyragardurinn,2024-11-18,http://api.waqi.info/feed/iceland/husdyrag.


In [83]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 176.0+ bytes


In [85]:
### Get the last mesurement - from yesterday 

aq_yesterday_df = air_quality_with_lag_fg.read().sort_values(by='date').tail(1)
aq_yesterday_df



Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.33s) 


,date,pm25,pm25_lag_1,pm25_lag_2,pm25_lag_3,country,city,street,url
1448,2024-11-18 00:00:00+00:00,11.0,13.0,23.0,27.0,iceland,reykjavik,husdyragardurinn,http://api.waqi.info/feed/iceland/husdyrag.


In [88]:
### then add the lag - based on data from prevoius day 
aq_today_df['pm25_lag_1'] = aq_yesterday_df.iloc[0]['pm25']
aq_today_df['pm25_lag_2'] = aq_yesterday_df.iloc[0]['pm25_lag_1']
aq_today_df['pm25_lag_3'] = aq_yesterday_df.iloc[0]['pm25_lag_2']
aq_today_df

,pm25,country,city,street,date,url,pm25_lag_1,pm25_lag_2,pm25_lag_3
0,22.0,iceland,reykjavik,husdyragardurinn,2024-11-18,http://api.waqi.info/feed/iceland/husdyrag.,11.0,13.0,23.0


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [89]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 64.0°N -22.25°E
Elevation 0.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-18,-2.70,0.0,22.539564,26.564985,reykjavik
1,2024-11-19,-3.70,0.0,26.474077,22.380184,reykjavik
2,2024-11-20,-4.05,0.0,27.009598,21.092421,reykjavik
3,2024-11-21,-7.45,0.0,8.121970,77.195740,reykjavik
4,2024-11-22,-5.70,0.0,8.287822,55.619598,reykjavik
5,2024-11-23,-4.40,0.0,9.178235,64.440094,reykjavik
6,2024-11-24,-6.50,0.0,10.495713,22.166399,reykjavik
7,2024-11-25,-4.20,0.0,7.704336,142.594574,reykjavik
8,2024-11-26,0.05,0.0,15.990646,148.815079,reykjavik
9,2024-11-27,3.35,0.3,33.518684,114.087448,reykjavik


In [8]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 452.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [90]:
# Insert new data
# air_quality_fg.insert(aq_today_df)
air_quality_with_lag_fg.insert(aq_today_df)

2024-11-18 15:59:14,350 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1159325/fs/1150028/fg/1354085


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_with_3day_lag_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1159325/jobs/named/air_quality_with_3day_lag_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x294254d1510>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 677940
         }
       },
       "result": {
         "observed_value": 22.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-18T02:59:14.000349Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsu

In [91]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-18 15:59:31,549 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1159325/fs/1150028/fg/1339780


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1159325/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x29425264550>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 663576
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-18T02:59:31.000549Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_betwe

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
